In [44]:
import pandas as pd
import requests
# computer 
bpi_tables = pd.read_html("https://www.espn.com/mens-college-basketball/bpi")
# print(bpi_tables[0][''])
bpi_tables[1]['Team'] = bpi_tables[0]['Team']
bpi_tables[1]['CONF'] = bpi_tables[0]['CONF']

bpi_tables[1].to_csv('bpi.csv')
# human rankings
# sagarin_table = pd.read_html("http://www.usatoday.com/sports/ncaab/sagarin/")
# ncaa_table = pd.read_html("")


In [43]:
print(bpi_table[1])

   Unnamed: 0_level_0 POWER INDEX                          PROJECTIONS  \
                  W-L         BPI BPI RK TREND   OFF   DEF     OVR W-L   
0                28-3        23.3      1    --   9.4  13.9    28.0-3.0   
1                28-3        19.8      2    --  13.3   6.5    28.0-3.0   
2                28-3        19.6      3    --  12.4   7.2    28.0-3.0   
3                24-7        18.7      4    --  11.3   7.4    24.0-7.0   
4                24-7        18.3      5    --   7.9  10.4    24.0-7.0   
5                24-7        18.0      6    --   9.8   8.2    24.0-7.0   
6                24-7        17.3      7     1  11.2   6.1    24.0-7.0   
7                23-8        16.8      8     2  10.9   5.9    23.0-8.0   
8               21-10        16.7      9     2  13.2   3.5   21.0-10.0   
9                24-7        16.7     10     1   5.7  11.0    24.0-7.0   
10               25-6        16.3     11     1   8.1   8.1    25.0-6.0   
11               23-8        15.6     

In [22]:
# from bs4 import BeautifulSoup

pom_table = pd.read_html("pomeroy.html")
print(pom_table)

[     Unnamed: 0_level_0      Unnamed: 1_level_0  Unnamed: 2_level_0  \
                     Rk                    Team                Conf   
     Unnamed: 0_level_2      Unnamed: 1_level_2  Unnamed: 2_level_2   
                     Rk                    Team                Conf   
     Unnamed: 0_level_4      Unnamed: 1_level_4  Unnamed: 2_level_4   
                     Rk                    Team                Conf   
     Unnamed: 0_level_6      Unnamed: 1_level_6  Unnamed: 2_level_6   
                     Rk                    Team                Conf   
     Unnamed: 0_level_8      Unnamed: 1_level_8  Unnamed: 2_level_8   
                     Rk                    Team                Conf   
    Unnamed: 0_level_10     Unnamed: 1_level_10 Unnamed: 2_level_10   
                     Rk                    Team                Conf   
    Unnamed: 0_level_12     Unnamed: 1_level_12 Unnamed: 2_level_12   
                     Rk                    Team                Conf   
    U

In [26]:
lrmc_table = pd.read_html("lrmc.html")
print(lrmc_table)

[     Unnamed: 0  Through games of 3/11/2024 Through games of 3/11/2024.1  \
0           NaN                   LRMC Rank                         Team   
1           NaN                         NaN                          NaN   
2           NaN                           1                      Arizona   
3           NaN                           2                      Houston   
4           NaN                           3                    Tennessee   
..          ...                         ...                          ...   
431         NaN  Through games of 3/11/2024   Through games of 3/11/2024   
432         NaN                   LRMC Rank                         Team   
433         NaN                         NaN                          NaN   
434         NaN                         361                    Coppin_St   
435         NaN                         362                 MS_Valley_St   

    Through games of 3/11/2024.2  Unnamed: 4  \
0                     Conference      

In [29]:
moore_table = pd.read_csv("moore.csv")
print(moore_table)

      RK                    TEAM   W   L  T    SOS     PR
0      1                 HOUSTON  28   3  0  79.19  95.51
1      2             CONNECTICUT  28   3  0  77.50  93.63
2      3                  PURDUE  28   3  0  79.82  92.32
3      4                 ARIZONA  24   7  0  78.57  92.26
4      5                  AUBURN  24   7  0  77.69  91.82
..   ...                     ...  ..  .. ..    ...    ...
357  358              COPPIN ST.   2  26  0  66.35  52.18
358  359                 PACIFIC   6  26  0  64.64  51.64
359  360                     VMI   4  28  0  64.63  51.20
360  361                   IUPUI   6  26  0  63.43  50.69
361  362  MISSISSIPPI VALLEY ST.   1  30  0  67.77  49.01

[362 rows x 7 columns]
